In [7]:
# Run this cell only if you are using Google Colab. 

# install required dependencies
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !pip install -q pyomo
  !apt-get install -y -qq glpk-utils
  !apt-get install -y -qq coinor-cbc
  !wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
  !unzip -o -q ipopt-linux64
  !pip install ttictoc

In [8]:
def DugoffLinearizer(slip_angle,slip_ratio, ca, cs, mu, Fz):
  lamb_bar = mu*Fz*(1+slip_ratio)/(2*((cs*slip_ratio)**2+(ca*slip_angle)**2)**0.5)
  if slip_angle !=0 or slip_ratio !=0:
    if lamb_bar < 1:
      fbar = lamb_bar*(2-lamb_bar)
      c_bar = (Fz*ca*mu)/(2*(slip_angle**2*ca**2 + cs**2*slip_ratio**2)**(1/2)) - (Fz*slip_angle**2*ca**3*mu)/(2*(slip_angle**2*ca**2 + cs**2*slip_ratio**2)**(3/2))

    else:
      fbar = 1
      c_bar = ca/(1+slip_ratio)
    Fy_bar= ca*slip_angle*fbar/(1+slip_ratio)
    c_bar = c_bar.item()
    Fy_bar = Fy_bar.item()
  else:
    fbar = 1
    c_bar = ca
    Fy_bar = 0

  return Fy_bar, c_bar, slip_angle

In [9]:
import numpy as np
from scipy.linalg import block_diag
import sympy as sym
import pyomo.environ as pyo
# States: y,v,psi, r, phi, phi_dot
# Inputs: Change in each wheel torque and in each wheel steer angle
def CFTOC_VD(slip_angle, slip_ratio, ca, cs, mux,muy, fz0,fy0, u,reff,xd,tf,tr,lf,lr,ms,hs,m,ixx,ks,ls,g,bs,iz,kus_d,Ts,Q,R,W0,Phir,alphamax,Qmax ,Qmin, deltamax,x0,psid,yd,N):
  """
  STATES:
  y: lateral position
  vy: lateral speed
  psi: heading angle
  r: yaw rate
  phi: roll angle
  phid: roll rate
  
  =================================================================
  =================================================================
  Input Definitions:

  slip_angle: tire slip angle at each tire (aka alpha) --- (5x1)
  slip_ratio: tire slip ratio at each tire (aka sigma) --- (5x1)
  ca: tire cornering stifness (aka C_alpha)            --- (5x1)
  cs: tire longitudinal stiffness (aka C_sigma)        --- (5x1)
  mux: coefficient of friction in x                    --- (scalar)
  muy: coefficient of friction in y                    --- (scalar)
  fz0: tire vertical load                              --- (5x1)
  fyo: tire lateral load                               --- (5x1)
  u: longitudinal velcoity                             --- (scalar)
  reff: tire effective radius of rotation              --- (scalar)
  delta1: initial commanded steering input             --- (scalar)
  tf: front wheel track (dist between tire centers)    --- (scalar)
  tr: rear wheel track (dist between tire centers)     --- (scalar)
  lf: distance from cg to front wheels                 --- (scalar)
  lr: distance from cg to rear wheels                  --- (scalar)
  ms: vehicle sprung mass (without mass of tires and suspension) --- (scalar)
  hs: cg of sprung mass to roll center                 --- (scalar)
  m: total mass of vehicle (sprung and unsprung)       --- (scalar)
  ixx: moment of inertia about the x-axis (roll)       --- (scalar)
  ks: spring stiffness of suspension                   --- (scalar)
  ls: distance between suspension fixtures to vehicle  --- (scalar)
  g: gravity                                           --- (scalar)
  bs: suspension damping coefficient                   --- (scalar)
  iz: vehivle moment of inertia about z axis (yaw)     --- (scalar)
  kus_d: desired understeer coefficient of vehicle     --- (scalar)
  rmax: maximum allowed yaw rate (to be selected)      --- (scalar)
  Ts: sampling time for euler discretization           --- (scalar)
  Q: cost matrix for states                            --- (5x5)
  R: cost matrix for inputs                            --- (5x5)
  W0: initial torques and steering angles [Q1,delta1,Q2,delta2,Q3,delta3,Q4,delta4] ---(8x1)
  Phir: banking angle at over the prediction horizon   --- (Nx1)
  alphamax: maximum allowable rear wheel side slip     --- (scalar)
  Qmax: maximum torque attainable at each wheel        --- (scalar)
  deltamax: maximum steering angle                     --- (scalar)
  x0: initial states [y,vy,r,phi,phid]                 --- (5x1)
  psid: desired heading angle                          --- (scalar)
  yd:desired lateral position                          --- (scalar)
  N: MPC Prediction horizon                            --- (scalar)
  """
  psi_desired = np.arctan(yd/xd)
  if u != 0:
    delta1 = psi_desired + x0[1]/u
  else: 
    delta1 = psi_desired
  # Tire model Linearization 
  fy1, ca1, a1 = DugoffLinearizer(slip_angle[0],slip_ratio[0], ca[0], cs[0], mux, fz0[0])
  fy2, ca2, a2 = DugoffLinearizer(slip_angle[1],slip_ratio[1], ca[1], cs[1], mux, fz0[1])
  fy3, ca3, a3 = DugoffLinearizer(slip_angle[2],slip_ratio[2], ca[2], cs[2], mux, fz0[2])
  fy4, ca4, a4 = DugoffLinearizer(slip_angle[3],slip_ratio[3], ca[3], cs[3], mux, fz0[3])  
  B11 = np.array([[0,0,0,0,0], [0,-ca1/u, -lf*ca1/u, 0, 0]])
  B21 = np.array([[1/reff, 0],[0, ca1]])
  D11 = np.array ([0, fy1-ca1*a1])

  B12 = np.array([[0,0,0,0,0], [0,-ca2/u, -lf*ca2/u, 0, 0]])
  B22 = np.array([[1/reff, 0],[0, ca2]])
  D12 = np.array ([0, fy2-ca2*a2])

  B13 = np.array([[0,0,0,0,0], [0,-ca3/u, lr*ca3/u, 0, 0]])
  B23 = np.array([[1/reff, 0],[0, ca3]])
  D13 = np.array ([0, fy3-ca3*a3])

  B14 = np.array([[0,0,0,0,0], [0,-ca4/u, lr*ca4/u, 0, 0]])
  B24 = np.array([[1/reff, 0],[0, ca4]])
  D14 = np.array ([0, fy4-ca4*a4])
  
  #Local actuator reconfiguration matrix: 0 inactive actuator 1 active actuator
  tq1 = 0
  tq2 = 0
  tq3 = 0
  tq4 = 0
  tdelta1 = 1
  tdelta2 = 1
  tdelta3 = 0
  tdelta4 = 0

  Tw1 = np.array([[tq1,0],[0,tdelta1]])
  Tw2 = np.array([[tq2,0],[0,tdelta2]])
  Tw3 = np.array([[tq3,0],[0,tdelta3]])
  Tw4 = np.array([[tq4,0],[0,tdelta4]])

  #Reconfiguration (Mapping) Matricies: Delta1 input
  Lw1 = np.array ([[np.cos(delta1), -np.sin(delta1)],[np.sin(delta1), np.cos(delta1)]])
  Lw2 = np.array ([[np.cos(delta1), -np.sin(delta1)],[np.sin(delta1), np.cos(delta1)]])
  Lw3 = np.array([[1,0],[0,1]])
  Lw4 = np.array([[1,0],[0,1]])

  #Mapping Matrix from corner forces
  Lc = np.array([[1,0,1,0,1,0,1,0],[0,1,0,1,0,1,0,1], [-tf/2,lf,tf/2,lf,-tr/2,-lr,tr/2,-lr]])

  # Vehicle Body Dynamics y vy psi,psidot roll, rollrate 
  # Equations written in block 1 below
  den =2*(ms**2*hs**2+m*ms*hs**2+m*ixx)
  Af = np.array([[0,1,0,0,0], [0,0,-u, (hs*ks*ls**2*ms-2*g*hs**2*ms**2)/den, bs*hs*ls**2*ms/den],[0,0,0,0,0],[0,0,0,0,1],[0,0,0,(2*g*hs*m*ms-ks*ls**2*m)/den, -bs*ls*ls*m/den]])
  Bf = np.array([[0,0,0],[0,(ixx+ms*hs**2)/den, 0],[0,0,1/iz],[0,0,0],[0,2*ms*hs/den,0]])
  Cpr = np.array([[0],[(2*g*ms**2*hs**2-hs*ks*ls**2*ms)/den],[0],[0],[(ks*ls**2*m-2*g*hs*m*ms)/den]])
  
  # X_dot = AX+EW+BU+D+FPhir
  # W = wheel torque, steer angle for each wheel (8x1)
  # States are y v r phi, phi_dot (ignoring tyre dynamics)
  # U = change in wheel torque, change in steering angle for each tire

  Tw = block_diag(Tw1,Tw2,Tw3,Tw4)
  Lw = block_diag(Lw1, Lw2, Lw3, Lw4)
  B1 = np.hstack((B11.T, B12.T, B13.T, B14.T)).T
  B2 = block_diag(B21, B22, B23, B24)
  D1 = np.hstack((D11,D12,D13,D14)).T
  
  A = Af+Bf@Lc@Lw@B1
  E = Bf@Lc@Lw@B2
  B = Bf@Lc@Lw@B2@Tw
  D = Bf@Lc@Lw@D1
  D = D.reshape(5,1)
  F = Cpr

###############################################################################
  ## CONTROL OBJECTIVES ##
  rmax = muy*g/u
  l= lf+lr
  rb = u/(l+kus_d*u**2)*delta1
  rd = np.sign(delta1)*np.min([np.abs(rb), rmax]).item()
  xd = np.array([yd,0, rd, 0 ,0])
  xd = xd.reshape(5,1)
  nX = 5
  nU = 8
  nW = 8
  nphir = 1
  # In the paper they use zero order hold, I will use euler discretization
  Ad = Ts*A + np.eye(np.size(A,0))
  Bd = Ts*B
  Ed = Ts*E
  Dd = Ts*D
  Fd = Ts*F

  model = pyo.ConcreteModel()
  
  model.tidx = pyo.Set(initialize=range(0, N+1))
  model.xidx = pyo.Set(initialize=range(0, nX))
  model.uidx = pyo.Set(initialize=range(0, nU))

  model.x = pyo.Var(model.xidx, model.tidx)
  model.u = pyo.Var(model.uidx, model.tidx)
  model.phir = phir
  model.xd = xd
  model.Q = Q
  model.R = R
  model.N = N
  model.A = Ad
  model.B = Bd
  model.E = Ed
  model.D = Dd
  model.F = Fd
  model.W0 = W0
  model.P = Q
  
  #Objective:
  def objective_rule(model):
    costX = 0.0
    costU = 0.0
    costT = 0.0
    for t in model.tidx:
      for i in model.xidx:
        for j in model.xidx:
          if t < N:
            costX += model.x[i,t] *model.Q[i,j]*model.x[j,t]
    for t in model.tidx:
      for i in model.uidx:
        for j in model.uidx:
          if t < N:
            costU += model.u[i,t]*model.R[i,j]*model.u[j,t]
    for i in model.xidx:
      for j in model.xidx:
        costT += model.x[i,model.N]*model.P[i,j]*model.x[j,model.N]
    return costX+costU+costT
  model.Obj = pyo.Objective(rule = objective_rule, sense = pyo.minimize)

  #Model Dynanamics
  def eq_const_rule(model,i,t):
    return model.x[i,t+1]-(sum(model.A[i,j]*model.x[j,t] for j in model.xidx)+ sum(model.E[i,j] * model.W0[j] for j in model.uidx)+ model.F[i] * model.phir +sum(model.B[i,j] * model.u[j,t] for j in model.uidx)+model.D[i]) == 0 if t < model.N else pyo.Constraint.Skip

  model.constraint1 = pyo.Constraint(model.xidx, model.tidx, rule = eq_const_rule)

  # Input Constraints:
  # Torque Saturation:
  model.constraint2 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[0,t] <= Qmax - model.W0[0] if t < N else pyo.Constraint.Skip)
  model.constraint3 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[2,t] <= Qmax - model.W0[2] if t < N else pyo.Constraint.Skip)
  model.constraint4 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[4,t] <= Qmax - model.W0[4] if t < N else pyo.Constraint.Skip)
  model.constraint5 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[6,t] <= Qmax - model.W0[6] if t < N else pyo.Constraint.Skip)
  
  model.constraint6 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[0,t] >= Qmin - model.W0[0] if t < N else pyo.Constraint.Skip)
  model.constraint7 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[2,t] >= Qmin - model.W0[2] if t < N else pyo.Constraint.Skip)
  model.constraint8 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[4,t] >= Qmin - model.W0[4] if t < N else pyo.Constraint.Skip)
  model.constraint9 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[6,t] >= Qmin - model.W0[6] if t < N else pyo.Constraint.Skip)

  
  fx0_p=np.zeros((4,1))
  for i in range(4):
    fx0_p[i] = mux*fz0[i]*np.sqrt(1-(fy0[i]/(muy*fz0[i]))**2)
  
  # Friction Circle
  model.constraint10 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[0,t] <= fx0_p[0].item() - model.W0[0] if t < N else pyo.Constraint.Skip)
  model.constraint11= pyo.Constraint(model.tidx, rule = lambda model, t: model.u[2,t] <= fx0_p[1].item() - model.W0[2] if t < N else pyo.Constraint.Skip)
  model.constraint12 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[4,t] <= fx0_p[2].item() - model.W0[4] if t < N else pyo.Constraint.Skip)
  model.constraint13 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[6,t] <= fx0_p[3].item() - model.W0[6] if t < N else pyo.Constraint.Skip)

  model.constraint14 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[0,t] >= -fx0_p[0].item() - model.W0[0] if t < N else pyo.Constraint.Skip)
  model.constraint15 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[2,t] >= -fx0_p[1].item() - model.W0[2] if t < N else pyo.Constraint.Skip)
  model.constraint16 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[4,t] >= -fx0_p[2].item() - model.W0[4] if t < N else pyo.Constraint.Skip)
  model.constraint17 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[6,t] >= -fx0_p[3].item() - model.W0[6] if t < N else pyo.Constraint.Skip)

  # Maximum steering angle constraints
  model.constraint18 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[1,t] <= deltamax - model.W0[1] if t < N else pyo.Constraint.Skip)
  model.constraint19= pyo.Constraint(model.tidx, rule = lambda model, t: model.u[3,t] <= deltamax - model.W0[3] if t < N else pyo.Constraint.Skip)
  model.constraint20 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[5,t] <= deltamax - model.W0[5] if t < N else pyo.Constraint.Skip)
  model.constraint21 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[7,t] <= deltamax - model.W0[7] if t < N else pyo.Constraint.Skip)

  model.constraint22 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[1,t] >= -deltamax - model.W0[1] if t < N else pyo.Constraint.Skip)
  model.constraint23 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[3,t] >= -deltamax - model.W0[3] if t < N else pyo.Constraint.Skip)
  model.constraint24 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[5,t] >= -deltamax - model.W0[5] if t < N else pyo.Constraint.Skip)
  model.constraint25 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[7,t] >= -deltamax - model.W0[7] if t < N else pyo.Constraint.Skip)

  ## State Constraints
  model.constraint26 = pyo.Constraint(model.tidx, rule = lambda model, t: model.x[2,t] <= rmax if t <= N else pyo.Constraint.Skip)
  model.constraint27 = pyo.Constraint(model.tidx, rule = lambda model, t: model.x[2,t] >= -rmax if t <= N else pyo.Constraint.Skip)
  model.constraint28 = pyo.Constraint(model.tidx, rule = lambda model, t: model.x[1,t]/u +lr/u*model.x[2,t] <= alphamax if t<= N else pyo.Constraint.Skip)
  model.constraint29 = pyo.Constraint(model.tidx, rule = lambda model, t: model.x[1,t]/u +lr/u*model.x[2,t] >= -alphamax if t<= N else pyo.Constraint.Skip)
  model.constraint30 = pyo.Constraint(model.xidx, rule = lambda model, i: model.x[i,0] == x0[i])
  
  model.constraint31 = pyo.Constraint(model.xidx, rule = lambda model, i: model.x[0,N] == xd[0][0])

  
  
  results = pyo.SolverFactory('ipopt').solve(model).write()

  # Plotting
  # plot results
  x1 = [pyo.value(model.x[0,0])]
  x2 = [pyo.value(model.x[1,0])]
  x3 = [pyo.value(model.x[2,0])]
  x4 = [pyo.value(model.x[3,0])]
  x5 = [pyo.value(model.x[4,0])]

  u1 = [pyo.value(model.u[0,0])]
  u2 = [pyo.value(model.u[1,0])]
  u3 = [pyo.value(model.u[2,0])]
  u4 = [pyo.value(model.u[3,0])]
  u5 = [pyo.value(model.u[4,0])]
  u6 = [pyo.value(model.u[5,0])]
  u7 = [pyo.value(model.u[6,0])]
  u8 = [pyo.value(model.u[7,0])]

  for t in model.tidx:
      if t < N:
          x1.append(pyo.value(model.x[0,t+1]))
          x2.append(pyo.value(model.x[1,t+1]))
          x3.append(pyo.value(model.x[2,t+1]))
          x4.append(pyo.value(model.x[3,t+1]))
          x5.append(pyo.value(model.x[4,t+1]))
      if t < N-1:
          u1.append(pyo.value(model.u[0,t+1]))
          u2.append(pyo.value(model.u[1,t+1]))
          u3.append(pyo.value(model.u[2,t+1]))
          u4.append(pyo.value(model.u[3,t+1]))
          u5.append(pyo.value(model.u[4,t+1]))
          u6.append(pyo.value(model.u[5,t+1]))
          u7.append(pyo.value(model.u[6,t+1]))
          u8.append(pyo.value(model.u[7,t+1]))


  tgrid = np.arange(0,N+1)*Ts
  uOpt = np.vstack((u1,u2,u3,u4,u5,u6,u7,u8))
  xOpt = np.vstack((x1,x2,x3,x4,x5))

  return xOpt, uOpt

In [10]:
# States: y,v,psi, r, phi, phi_dot
# Inputs: Change in each wheel torque and in each wheel steer angle

# Vehicle Parameters: 
reff = 0.1905
lf = 1.6566
lr = 1.3152
l = lf+lr
tf = 1.638762
tr = 1.5239686
m = 803.182 #kg
mf = 782/2.2
mr = 985/2.2
mufw = 29.5
murw = 36
ms = m - 2*(mufw+murw)
ixx = 200 
hs = 0 #considered CG and roll center to be same point ##############
iy = 800
iz = 1200
g = 9.81

cs = 80000*np.ones((4,1)) ####
ca = 47275*np.ones((4,1)) #####
x0 = np.array([0,0,0.1,0,0])
slip_angle = np.array([0,0,0,0])
slip_ratio = np.array([0.05,0.05,0.05,0.05])
fz0 = m*g/4*np.ones((4,1))
W0 = np.array([0,0,0,0,0,0,0,0])
fy0 = np.zeros((4,1))
kus_d = mf /ca[0] +mr /cs[0] #Caf and Car needed
mux = 1
muy =1
u = 20
xd = 100
ks = 10000
ls =1
bs = 1000
Ts = 0.1
Q =np.eye(5)
R = np.eye(8)
phir = 0
alphamax = 0.2
Qmax = 1000
deltamax = 0.2
psid = 0.2
yd = 5
N = 100
#################################################################################
nx = 6
nu = 8

M = 1   # Simulation time

xOpt = np.zeros((nx, M+1))
uOpt = np.zeros((nu, M))


xPred = np.zeros((nx, N+1, M))

feas = np.zeros((M, ), dtype=bool)

for t in range(M):
    uopt,jopt = CFTOC_VD(slip_angle, slip_ratio, ca, cs, mux,muy, fz0,fy0, u,reff,xd,tf,tr,lf,lr,ms,hs,m,ixx,ks,ls,g,bs,iz,kus_d,Ts,Q,R,W0,phir,alphamax,Qmax,-2000,deltamax,x0,psid,yd,N)    
      
    JO = jopt
    print(uopt)
    print(np.shape(uopt))


    ipopt


ApplicationError: No executable found for solver 'ipopt'